<h1> Some simulations and properties of Galton Watson process</h1>

<h2> Introduction </h2>

<p> This project is about studying simple properties of Galton Watson process. This process can be used in order to solve problems about the development of a population under some conditions. Galton, for example, used it to study the extinction of the names of nobles in the past centuries. <p>

<p> We will basically propose a model to answer this similar problem : Considering that Mr.Jerry Smith is the last male named Smith in the world, what is the probability of the survivance of the Smith male in the next generation ? Can we guess the number of Smith male in ten generations ? <p>

<p> This project needs only basic maths, basic Python library and Monte Carlo simulations, and it was my introduction to the Python language.  <p>

<h2> Definition of the model </h2>

<p> We consider the random variable $(X_n)$ which represents the number of male descendants of Mr. Smith after n generations. So, $X_0=1$ as Jerry Smith is the only Smith male at the beginning, $X_1$ represents the number of sons of Mr. Smith, $X_2$ is grandsons...<p>
* Asumption 1 : $(X_n)$ is a Markov Chain, you only need to know $ X_{n-1}$ to predict $X_n$, even if you have informations about every previous $X_k$ 
* Asumption 2 : we know the probability for a male of every generation $n$ to have $k$ sons, for every $k \in N$. We note this probability $p_k^n$, $(k,n) \in N^2$
* Asumption 3 : Independance between the number of sons of every member of the family

Basically, this means that in our model we do not consider collective behaviours : your probability of having no child is the same if your brothers already have 1 son or 10 sons. We also first consider that all of the laws are known at the beginning of the experience, and that the fact that your father and your grand-father imply no change in your law of probability.

This leads to the following process :

* $ \forall n \in N $ , $ X_{n+1}=\sum_{k=1}^{X_n} Y_k$ , where $ \forall k $ , $Y_k$ is a random variable following $(P_k^n)_{k \in N}$

## First samplings

I first consider that $(p_k^n)$ is constant over the time, and we take the probability law from the french National Institute of Statistics and Economic Studies (Insee). With our data the average number of sons of our individuals will be 1.023. 

I first import the libraries and the law of probability.

In [387]:
import numpy as np
import numpy.random as rd
import scipy.stats as sp
import matplotlib.pyplot as plt

P = np.array([0.312,0.416,0.215,0.051,0.006])
Values = np.array([0,1,2,3,4])
nb_mean = sum(P*Values)
nb_sd=np.sqrt(sum(P*Values**2))

print(nb_mean,nb_sd)

def nb_sons_INSEE(nb_fathers) :
    return (rd.choice(Values, p=P, size=nb_fathers))

Probability_law=nb_sons_INSEE

1.023 1.353144486


I first implement a class which allows to create a new generation knowking the informations about the previous generation. I choose to create objects because it seems the easiest way to keep at every time the most informations if I choose to analyze the properties of my process in a second time. 

In [388]:
class generation :
    def __init__(self,previous_gen):
        self.previous_gen = previous_gen
        self.nb_fathers = 1
        if previous_gen != 0 :
               self.nb_fathers = previous_gen.nb_indiv
        self.law=Probability_law
        self.list_indiv=self.law(self.nb_fathers)
        self.nb_indiv=sum(self.list_indiv)

Jerry_sons = generation(0)
Jerry_sons

In [394]:
print("number of sons of Jerry:", Jerry_sons.nb_indiv)

gen1 = generation(Jerry_sons)
print("grandsons :", gen1.list_indiv, "number of grandsons : ", gen1.nb_indiv)

number of sons of Jerry: 2
grandsons : [3 1] number of grandsons :  4


I know create a class for the sampling of a complete genealogy tree. I add some interesting statistics in the properties of the class.

In [182]:
class Simulation :
    def __init__(self,n):
        self.nb_gen=n
        self.generations=[generation(0)]
        self.generations=self.get_generations(self.generations)
        self.nb_by_gen=self.get_list_nb_indiv()
        self.Is_extinct=(0 in self.nb_by_gen)
        self.extinction_time="Not extinct"
        if self.Is_extinct :
            self.extinction_time=self.nb_by_gen.index(0)
        
    def get_generations(self,G):
        for i in range(self.nb_gen-1):
            G.append(generation(G[-1]))
        return(G)
    
    def get_list_nb_indiv(self):
        L=[1]
        for x in self.generations :
            L.append(x.nb_indiv)
        return(L)
    
    def graph(self):
        colors=["red","blue","green","yellow","purple","navy","pink","orange"]
        plt.plot(self.nb_by_gen,color=rd.choice(colors))
    

In [202]:
Results=[]
for x in range(50) :
    S=Simulation(20)
    S.graph()
    Results.append([S.Is_extinct,S.extinction_time])
plt.show()
print(Results)

[[True, 9], [True, 1], [True, 11], [True, 4], [True, 1], [True, 5], [True, 1], [True, 2], [True, 2], [True, 1], [True, 1], [True, 5], [True, 1], [True, 15], [True, 1], [True, 1], [True, 12], [True, 2], [True, 1], [True, 4], [False, 'Not extinct'], [True, 7], [True, 2], [True, 2], [True, 4], [True, 1], [True, 12], [True, 1], [True, 4], [True, 3], [True, 7], [False, 'Not extinct'], [False, 'Not extinct'], [True, 4], [True, 2], [True, 3], [True, 2], [True, 8], [False, 'Not extinct'], [False, 'Not extinct'], [True, 3], [True, 1], [True, 5], [True, 2], [True, 6], [False, 'Not extinct'], [False, 'Not extinct'], [True, 13], [True, 2], [True, 3]]


Now that I have a class for simulations, I create a new class for Monte Carlo Simulation. What can I observe if I sample a lot of genealogy trees ?

In [306]:
class MC_simul :
    def __init__(self,N,n):
        self.nb_simul=N
        self.nb_gen=n
        self.Simul=self.get_simul(N,n)
        self.Extinct_rate=sum(x.Is_extinct for x in self.Simul)/self.nb_simul
        self.Extinct_times=[x.extinction_time for x in self.Simul]
        self.list_times = list(filter(lambda x: x != "Not extinct", self.Extinct_times))
        self.mean_times = np.mean(self.list_times)
        self.sd_times = np.std(self.list_times)
    
    def get_simul(self,N,n):
        S=[]
        for x in range(N):
            S.append(Simulation(n))
        return(S)
    
    def plot(self):
        for x in self.Simul:
            x.graph()
        plt.show()
    

In [309]:
MC_sample=MC_simul(1000,30)
print(MC_sample.Extinct_rate)
print(MC_sample.list_times)
print(MC_sample.mean_times,MC_sample.sd_times)
MC_sample.plot()


0.884
[1, 16, 2, 1, 4, 25, 1, 9, 2, 5, 11, 10, 1, 4, 2, 5, 12, 2, 1, 2, 8, 9, 14, 8, 2, 5, 1, 20, 2, 5, 1, 12, 3, 4, 1, 7, 1, 6, 1, 2, 1, 9, 3, 28, 1, 7, 4, 1, 1, 5, 1, 8, 4, 2, 8, 24, 2, 2, 5, 6, 1, 5, 3, 2, 1, 8, 1, 1, 1, 1, 1, 28, 4, 2, 7, 26, 1, 11, 30, 1, 1, 21, 4, 2, 3, 4, 1, 12, 3, 6, 1, 1, 7, 7, 1, 2, 3, 5, 1, 8, 20, 1, 1, 1, 1, 16, 3, 1, 10, 30, 3, 4, 1, 2, 10, 1, 1, 1, 2, 1, 5, 16, 1, 1, 4, 3, 1, 3, 2, 1, 3, 1, 1, 2, 10, 1, 6, 13, 1, 2, 7, 11, 20, 2, 2, 2, 4, 4, 6, 6, 1, 1, 6, 1, 1, 2, 3, 9, 1, 1, 4, 24, 3, 6, 2, 19, 4, 2, 1, 1, 1, 2, 1, 1, 2, 1, 4, 1, 1, 1, 15, 2, 1, 3, 1, 1, 1, 1, 7, 4, 5, 1, 5, 1, 4, 6, 2, 1, 8, 1, 2, 5, 2, 2, 2, 8, 1, 21, 2, 2, 1, 1, 1, 19, 2, 2, 1, 1, 6, 2, 5, 5, 4, 1, 1, 5, 3, 22, 2, 2, 3, 3, 1, 1, 1, 3, 5, 1, 14, 2, 1, 1, 25, 3, 1, 1, 12, 1, 10, 9, 9, 3, 17, 1, 2, 4, 2, 1, 1, 4, 5, 19, 1, 11, 3, 13, 1, 1, 1, 5, 4, 1, 3, 4, 7, 1, 3, 6, 1, 7, 1, 3, 3, 13, 2, 1, 28, 2, 4, 5, 1, 1, 7, 1, 1, 2, 1, 1, 1, 1, 5, 3, 2, 1, 6, 1, 3, 1, 1, 5, 1, 1, 1, 1, 1, 1, 1, 

If we consider our sample of extinction rate as a Monte Carlo estimator for the probability of the extinction of the Smith name, Jerry Smith has only 10% of chance to have a Smith descendant in 30 generations. The average extinction date is after 5 generations, but the standart deviation is very high.

Now we can do the same with different law of probability to check the behaviours of our process behind different probabilities. We can for example use a Poisson law with different means

In [395]:
def Poisson_sample(nb_fathers) :
    return (rd.poisson(lambd,size=nb_fathers))

Probability_law=Poisson_sample

lambd= 0.3

MC_sample=MC_simul(1000,30)
print(MC_sample.Extinct_rate)
print(MC_sample.mean_times,MC_sample.sd_times)
MC_sample.plot()

1.0
1.335 0.697692625731


In [396]:
lambd=1

MC_sample=MC_simul(1000,30)
print(MC_sample.Extinct_rate)
print(MC_sample.mean_times,MC_sample.sd_times)
MC_sample.plot()


0.947
4.37381203801 5.36322399637


In [399]:
lambd=2

MC_sample=MC_simul(1000,10)
print(MC_sample.Extinct_rate)
print(MC_sample.mean_times,MC_sample.sd_times)
MC_sample.plot()

0.202
1.58910891089 1.03142706006


Given a law of probability for the number of sons of every individual at every generation, it seems that we are able to give a numerical estimator of the probability of extinction of the family name Smith, and of the expectation and standard deviation we can check out the maths to verify these estimators.

## Mathematical properties